In [3]:
import scipy as sp
import spacy
import pandas as pd
import pickle
import gzip
from tqdm import tqdm
from google.cloud import bigquery
from logging import getLogger
from concurrent.futures import TimeoutError
import sys
import os
from pathlib import Path

PROJECT = 'reddit-network-184710'
CREDENTIALS = 'reddit-network-774059619c28.json'

CACHE = 'cache'

def client():
    return bigquery.Client.from_service_account_json(CREDENTIALS, project=PROJECT)

In [5]:
import pandas as pd

query = """SELECT subreddit, author, created_utc
            FROM `fh-bigquery.reddit_comments.2017_06`
            WHERE author IS 'changemyview'"""

query

"SELECT subreddit, author, created_utc\n            FROM `fh-bigquery.reddit_comments.2017_06`\n            WHERE author IS 'changemyview'"

In [13]:
import pandas as pd

query = """SELECT subreddit, author, created_utc
            FROM `fh-bigquery.reddit_comments.2017_06`
            WHERE subreddit IN ('changemyview', 'The_Donald')"""

In [14]:
config = bigquery.QueryJobConfig()
config.query_parameters = (bigquery.ScalarQueryParameter('size', 'INT64', 10),)

config = config or bigquery.QueryJobConfig()
config.use_legacy_sql = False
config.maximum_bytes_billed = int(5e9)

print('Submitting query')
j = client().query(query=query, job_config=config)
with tqdm() as pbar:
    while True:
        try:
            j.result(timeout=1)
        except TimeoutError:                
            pbar.update(1)
        else:
            break

Submitting query


8it [00:10,  1.31s/it]


In [15]:
print('Unpacking results')
    
total = j.query_results().total_rows

iterator = j.result()
rows = []
for row in tqdm(iterator, total=total):
    rows.append(row.values())

columns = [c.name for c in iterator.schema]
df = pd.DataFrame(rows, None, columns)

  0%|          | 0/1231681 [00:00<?, ?it/s]

Unpacking results


100%|██████████| 1231681/1231681 [01:05<00:00, 18676.24it/s]


In [16]:
df.shape

(1231681, 3)

In [17]:
df.to_csv('cmv-td-17-06-comments.csv')
df.to_pickle('cmv-td-17-06-comments.pkl')

In [20]:
df = pd.read_csv('cmv-td-17-06-comments.csv')
authors = list(df['author'].unique())
test_names = tuple(authors[:6])

authors = set(df['author'])

s = ''
for name in authors:
    s = s + "'" + name + "' ,"

s = s[:-2]

len(authors)


60286